In [12]:
from PIL import Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import keras
#from keras.utils import np_utils
import warnings
import joblib
warnings.filterwarnings("ignore")
#warnings.FutureWarning("ignore")
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [13]:
dir_path = "C:\cell_images\cell_images"

# Fetch directories
directories = [d for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d))]
directories

['Parasitized', 'Uninfected']

In [15]:
data = []
labels= []

In [16]:
parasitized_dir = "C:/cell_images/cell_images/Parasitized/"
parasitized = os.listdir(parasitized_dir)
for filename in parasitized:
    try:
        image_path = os.path.join(parasitized_dir, filename)
        image = cv2.imread(image_path)
        image_from_array = Image.fromarray(image, "RGB")
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(0)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")


uninfected_dir = "C:/cell_images/cell_images/Uninfected/"
uninfected = os.listdir(uninfected_dir)
for filename in uninfected:
    try:
        image_path = os.path.join(uninfected_dir, filename)
        image = cv2.imread(image_path)
        image_from_array = Image.fromarray(image, "RGB")
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(1)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

Error processing C:/cell_images/cell_images/Parasitized/Thumbs.db: 'NoneType' object has no attribute '__array_interface__'
Error processing C:/cell_images/cell_images/Uninfected/Thumbs.db: 'NoneType' object has no attribute '__array_interface__'


In [17]:
data =np.array(data)
labels =np.array(labels)

In [18]:
print(data.shape)
print(labels.shape)

(27558, 50, 50, 3)
(27558,)


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [20]:
X_train = X_train / 255
X_test = X_test / 255

In [33]:
from keras.utils import to_categorical
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

In [34]:
model = Sequential()

In [35]:
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(2,activation="softmax"))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 50, 50, 16)          │             208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 25, 25, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 25, 25, 32)          │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 12, 12, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 12, 12, 64)          │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 500)                 │       1,152,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │           1,002 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,164,046 (4.44 MB)

 Trainable params: 1,164,046 (4.44 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [37]:
history = model.fit(X_train, y_train_encoded, batch_size=50, epochs=10, validation_data=(X_test, y_test_encoded), verbose=1)

Epoch 1/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 23s 44ms/step - accuracy: 0.7052 - loss: 0.5306 - val_accuracy: 0.9410 - val_loss: 0.1725
Epoch 2/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.9477 - loss: 0.1634 - val_accuracy: 0.9550 - val_loss: 0.1488
Epoch 3/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.9555 - loss: 0.1413 - val_accuracy: 0.9557 - val_loss: 0.1410
Epoch 4/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.9562 - loss: 0.1248 - val_accuracy: 0.9537 - val_loss: 0.1590
Epoch 5/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step - accuracy: 0.9617 - loss: 0.1116 - val_accuracy: 0.9532 - val_loss: 0.1392
Epoch 6/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step - accuracy: 0.9636 - loss: 0.1057 - val_accuracy: 0.9565 - val_loss: 0.1336
Epoch 7/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.9698 - loss: 0.0875 - val_accuracy: 0.9563 - val_loss: 0.1249
Epoch 8/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.9712 - loss: 0.0808 - 

In [38]:
model.save('model_malaria.h5')

In [39]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image

In [40]:
model = load_model('model_malaria.h5')

In [50]:
img = image.load_img("C:\\cell_images\\Uninfected\\C1_thinF_IMG_20150604_104722_cell_242.png", target_size=(50,50))
x = image.img_to_array(img)
x = x / 255.0 

In [51]:
x

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)

In [52]:
from keras.applications.vgg16 import preprocess_input
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
img_data = np.squeeze(img_data)
img_data = img_data.reshape(-1, 50, 50, 3)

In [53]:
predictions = model.predict(img_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


In [54]:
predicted_class = np.argmax(predictions, axis=1)[0]

In [55]:
predicted_class

1

In [56]:
if predicted_class == 1:
    print("Uninfected")
else:
    print("Infected")

Uninfected
